# Text Analytics

自然言語処理 (NLP) は、書き言葉と話し言葉を扱う人工知能 (AI) の一分野です。NLP を使用して、テキストや音声から意味を抽出したり、意味のある応答を自然言語で作成したりするソリューションを構築できます。

Microsoft Azure *Cognitive Services* には、テキスト内のキー フレーズの識別や、感情に基づくテキストの分類など、すぐに使用できる NLP 機能を提供する *Text Analytics* サービスが含まれています。

![ノートを読んでいるロボット](./images/NLP.jpg)

たとえば、架空の旅行会社 *Margie's Travel* が、ホテル滞在のレビューを提出するよう顧客に勧めているとします。Text Analytics サービスを使用すると、キー フレーズを抽出してレビューを要約したり、どのレビューが肯定的でどれが否定的であるかを判断したり、レビューの文面を分析して場所や人などの既知の事項について言及しているか確認したりできます。

## レビュー ドキュメントを表示する

まず、顧客から寄せられたホテルのレビューをいくつか見てみます。

レビューはテキスト ファイルです。確認するには、セルの左側にある 「**セルの実行**」 (&#9655;) ボタンをクリックして、以下のコードを実行します。

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## Cognitive Services リソースを作成する

**Text Analytics** Cognitive Service を使用すると、これらのレビューのテキストを分析できます。このサービスを使用するには、Azure サブスクリプションで **Text Analytics** リソースまたは **Cognitive Services** リソースのいずれかをプロビジョニングする必要があります。使用する予定のサービスは Text Analytics だけである場合、または Text Analytics の使用状況を個別に追跡する場合は、Text Analytics リソースを使用します。それ以外の場合は、Cognitive Services リソースを使用すると、Text Analytics サービスを他の Cognitive Services と組み合わせることができ、開発者が単一のエンドポイントとキーを使用してそれらのサービスにアクセスできるようになります。

まだリソースを作成していない場合は、次の手順で Azure サブスクリプションに **Cognitive Services** リソースを作成します。

> **注**: Cognitive Services リソースが既にある場合は、Azure portal で**クイック スタート**ページを開き、キーとエンドポイントを以下のセルにコピーするだけで作成できます。それ以外の場合は、以下の手順に従って作成してください。

1. ブラウザーの新しいタブで Azure portal (https://portal.azure.com) を開き、Microsoft アカウントでサインインします。
2. 「**&#65291;リソースの作成**」 ボタンをクリックし、*Cognitive Services* を検索して、以下の設定で **Cognitive Services** リソースを作成します。
    - **サブスクリプション**: *使用する Azure サブスクリプション*
    - **リソース グループ**: *一意の名前のリソース グループを選択または作成します*
    - **リージョン**: *利用可能な任意のリージョンを選択します*。
    - **名前**: *一意の名前を入力します*。
    - **価格レベル**: S0
    - **注意事項を読み理解しました**: 選択されています。
3. デプロイが完了するまで待ちます。そのあと Cognitive Services リソースに移動し、「**概要**」 ページでリンクをクリックしてサービスのキーを管理します。クライアント アプリケーションから Cognitive Services リソースに接続するには、エンドポイントとキーが必要です。

### Cognitive Services リソースのキーとエンドポイントを取得する

Cognitive Services リソースを使用するには、クライアント アプリケーションにエンドポイントと認証キーが必要です。

1. Azure portalで、Cognitive Services リソースの 「**キーとエンドポイント**」 ページからリソースの 「**キー 1**」 の値をコピーし、以下のコードに貼り付けます (**YOUR_COG_KEY** と置き換える)。
2. リソースの**エンドポイント**をコピーして以下のコードに貼り付けます (**YOUR_COG_ENDPOINT** と置き換える)。
3. 以下のセルの緑色の <span style="color:green">&#9655;</span> ボタンをクリックして、コードを実行します。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## 言語を検出する
最初に、書かれているレビューの言語を識別します。

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## キー フレーズを抽出する

これで、顧客レビューのテキストを分析して、主要な論点を示すキー フレーズを識別できます。

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

キー フレーズは、各レビューで最も重要な論点を理解するのに役立ちます。たとえば、「スタッフが親切」や「サービスがよくない」というフレーズを含むレビューは、顧客の主要な懸念事項を示している可能性があります。

## 感情を判断する

*感情スコア*に基づいて、レビューを*肯定的*か*否定的*かで分類すると役立つ場合があります。この分類も、Text Analytics サービスを使用して行うことができます。

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5, 
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## 既知のエンティティを抽出する

*エンティティ*は、テキストの中で述べられている可能性のある事がらであり、一般的に理解されているタイプのものです。例えば、場所、人、日付などです。レビューに記載されている日付と場所に関心があるとします。その場合、次のコードを使用すると、それらのエンティティを見つけることができます。

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

エンティティの中には、ウィキペディアのページが関連付けられていて十分に知られているものもあります。そのような場合、Text Analytics サービスはウィキペディアのページの URL を返します。

## 詳細情報

Text Analytics サービスの詳細については、「[Text Analytics サービス ドキュメント](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)」を参照してください。